In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from globalModule.globalModule import *

In [11]:
__name__ = "RNN"

# Model

In [12]:
def getModel():
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    return model


In [13]:
def getData():
    labelMap = {label:num for num, label in enumerate(actions)}
    sequences, labels = [], []
    for action in actions:
        for sequence in np.array(os.listdir(os.path.join(gesturesDataPath, action))).astype(int):
            window = []
            for frame_num in range(sequenceLen):
                try:
                    res = np.load(os.path.join(gesturesDataPath, action, str(sequence), "{}.npy".format(frame_num)))
                    window.append(res)
                except Exception as e:
                    print(e)
            sequences.append(window)
            labels.append(labelMap[action])
    
    return sequences, labels 

In [14]:
sequences, labels = getData()
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [15]:
### INITS

tb_callback = TensorBoard(log_dir=log_dir)

In [16]:
model = getModel()
model.fit(X_train, y_train, epochs=400, callbacks=[tb_callback])

Epoch 1/400
6/6 [==============================] - 3s 30ms/step - loss: 1.3854 - categorical_accuracy: 0.2474
Epoch 2/400
6/6 [==============================] - 0s 29ms/step - loss: 1.3659 - categorical_accuracy: 0.3000
Epoch 3/400
6/6 [==============================] - 0s 30ms/step - loss: 1.2933 - categorical_accuracy: 0.3211
Epoch 4/400
6/6 [==============================] - 0s 31ms/step - loss: 1.1898 - categorical_accuracy: 0.4421
Epoch 5/400
6/6 [==============================] - 0s 30ms/step - loss: 1.1467 - categorical_accuracy: 0.5053
Epoch 6/400
6/6 [==============================] - 0s 28ms/step - loss: 1.1258 - categorical_accuracy: 0.5263
Epoch 7/400
6/6 [==============================] - 0s 29ms/step - loss: 1.2104 - categorical_accuracy: 0.4737
Epoch 8/400
6/6 [==============================] - 0s 28ms/step - loss: 1.1345 - categorical_accuracy: 0.5211
Epoch 9/400
6/6 [==============================] - 0s 28ms/step - loss: 0.9392 - categorical_accuracy: 0.5421
Epoch 10/4

In [17]:

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            48896     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                                 
 dense_7 (Dense)             (None, 4)                

In [18]:
model.save(f"{modelPath}/{__name__}/test.h5")